In [84]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import joblib

In [86]:
URL_data_sme_minmax_be = 'https://drive.google.com/file/d/1JV61ruhmwiLtUDsG6VdSLI_EdtlNvAuC/view?usp=share'
path_data_sme_minmax_be = 'https://drive.google.com/uc?export=download&id='+URL_data_sme_minmax_be.split('/')[-2]
data_sme_minmax_be = pd.read_csv(path_data_sme_minmax_be)

URL_data_sme_standard_be = 'https://drive.google.com/file/d/1kApVPeAqR4EPAp2Qqe5SGVcv3g13PjGE/view?usp=share'
path_data_sme_standard_be = 'https://drive.google.com/uc?export=download&id='+URL_data_sme_standard_be.split('/')[-2]
data_sme_standard_be = pd.read_csv(path_data_sme_standard_be)

URL_data_test_minmax_be = 'https://drive.google.com/file/d/1xzEL_joTrtUhRnUfu9x_5j7bgZJNlxqD/view?usp=share'
path_data_test_minmax_be = 'https://drive.google.com/uc?export=download&id='+URL_data_test_minmax_be.split('/')[-2]
data_test_minmax_be = pd.read_csv(path_data_test_minmax_be)

URL_data_test_standard_be = 'https://drive.google.com/file/d/19rJwtDQincWlBWw4pTS6r8Cvf11aKDfW/view?usp=share'
path_data_test_standard_be = 'https://drive.google.com/uc?export=download&id='+URL_data_test_standard_be.split('/')[-2]
data_test_standard_be = pd.read_csv(path_data_test_standard_be)


In [87]:
x_test_minmax_be = data_test_minmax_be.drop('level', axis=1)
y_test_minmax_be = data_test_minmax_be['level']

x_test_standard_be = data_test_standard_be.drop('level', axis=1)
y_test_standard_be = data_test_standard_be['level']

In [128]:
def DecisionTree(x_train, y_train, x_test, y_test):
    dt = DecisionTreeClassifier()

    # Define the hyperparameter search space
    max_depth = (10, 100)
    min_samples_split = (2, 10)
    min_samples_leaf = (1, 4)
    criterion = ['gini', 'entropy']

    search_space = {
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'criterion': criterion
    }

    # Define the hyperparameter search method
    dt_search = BayesSearchCV(
        dt, search_space, n_iter=100, cv=5, n_jobs=-1, random_state=42
    )

    dt_search.fit(x_train, y_train)
    # Select the hyperparameters that result in the best performance
    best_params = dt_search.best_params_

    print("Best parameters: ", best_params)

    dt = DecisionTreeClassifier(**best_params)

    dt.fit(x_train, y_train)

    # predict data train
    y_pred_train_labels = dt.predict(x_train)

    # Calculate the F1 score, recall, precision, and accuracy
    f1_train = f1_score(y_train, y_pred_train_labels, average='macro')
    recall_train = recall_score(y_train, y_pred_train_labels, average='macro')
    precision_train = precision_score(y_train, y_pred_train_labels, average='macro')
    accuracy_train = accuracy_score(y_train, y_pred_train_labels)
    
    print("\nEvaluation data training: ")
    print("Macro-Averaged Recall: ", f1_train)
    print("Macro-Averaged Precision: ", recall_train)
    print("Macro-Averaged Precision: ", precision_train)
    print("Accuracy: ", accuracy_train)

    # predict data test
    y_pred_labels = dt.predict(x_test)

    # Calculate the macro-averaged F1 score, recall, precision, and accuracy
    f1 = f1_score(y_test, y_pred_labels, average='macro')
    recall = recall_score(y_test, y_pred_labels, average='macro')
    precision = precision_score(y_test, y_pred_labels, average='macro')
    accuracy = accuracy_score(y_test, y_pred_labels)

    print("\nEvaluation data training: ")
    print("Macro-Averaged F1 Score: ", f1)
    print("Macro-Averaged Recall: ", recall)
    print("Macro-Averaged Precision: ", precision)
    print("Accuracy: ", accuracy)

    joblib.dump(dt, 'model_dt1.joblib')

## Data dengan kombinasi SMOTE-ENN

In [125]:
x_train_sme_minmax_be = data_sme_minmax_be.drop('level', axis=1)
x_train_sme_minmax_be.sample(6)

,precip,Libur,ROAD_CLOSED,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,...,street_1,street_2,street_3,street_4,street_5,street_6,street_7,street_8,street_9,street_10
89697,0.000163,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.000000,0.0,0.0,1.0,0.0
22858,0.000503,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.000000,1.0,0.0,1.0,1.0
10817,0.020453,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,1.0,1.0,1.0
48269,0.134984,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.281445,1.0,0.0,1.0,0.0
71797,0.004943,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.000000,1.0,1.0,0.0,1.0
94543,0.143171,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.000000,1.0,1.0,0.0,1.0


In [126]:
y_train_sme_minmax_be = data_sme_minmax_be['level']

In [129]:
DecisionTree(x_train_sme_minmax_be, y_train_sme_minmax_be, x_test_minmax_be, y_test_minmax_be)

/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best parameters:  OrderedDict([('criterion', 'entropy'), ('max_depth', 77), ('min_samples_leaf', 1), ('min_samples_split', 2)])

Evaluation data training: 
Macro-Averaged Recall:  1.0
Macro-Averaged Precision:  1.0
Macro-Averaged Precision:  1.0
Accuracy:  1.0

Evaluation data training: 
Macro-Averaged F1 Score:  0.5904822111455441
Macro-Averaged Recall:  0.6208380153144146
Macro-Averaged Precision:  0.5722677589425152
Accuracy:  0.7525406735633231


In [130]:
model_sme_minmax = joblib.load('model_dt1.joblib')

In [135]:
joblib.dump(model_sme_minmax, 'model_sme_minmax.joblib')

['model_sme_minmax.joblib']

## Data dengan kombinasi SMOTE-ENN

In [131]:
x_train_sme_standard_be = data_sme_standard_be.drop('level', axis=1)
x_train_sme_standard_be.sample(6)

,precip,Libur,ROAD_CLOSED,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,...,street_1,street_2,street_3,street_4,street_5,street_6,street_7,street_8,street_9,street_10
78512,-0.263531,-0.223265,-0.066881,-0.409020,2.464305,-0.409540,-0.409085,-0.407522,-0.408532,-0.408239,...,-0.890382,-0.886265,-0.904570,-0.992528,-0.995596,1.006938,1.004149,1.001838,-1.003287,-1.003757
32522,-0.276113,-0.223265,-0.066881,-0.409020,-0.405794,2.441761,-0.409085,-0.407522,-0.408532,-0.408239,...,-0.890382,-0.886265,-0.904570,0.859980,-0.995596,1.006938,-0.995868,-0.998166,-1.003287,-1.003757
74552,-0.131904,-0.223265,-0.066881,-0.409020,-0.405794,2.441761,-0.409085,-0.407522,-0.408532,-0.408239,...,-0.890382,1.128331,-0.904570,1.007528,1.004424,-0.993110,-0.995868,1.001838,0.996724,-1.003757
9403,-0.159896,-0.223265,-0.066881,-0.409020,-0.405794,-0.409540,-0.409085,-0.407522,2.447789,-0.408239,...,1.123114,-0.886265,-0.904570,-0.992528,1.004424,1.006938,1.004149,1.001838,-1.003287,0.996257
60696,-0.159780,-0.223265,-0.066881,-0.409020,-0.405794,-0.409540,2.444479,-0.407522,-0.408532,-0.408239,...,-0.890382,1.128331,-0.904570,1.007528,1.004424,-0.993110,1.004149,-0.998166,0.996724,-1.003757
11415,-0.275996,-0.223265,-0.066881,2.444868,-0.405794,-0.409540,-0.409085,-0.407522,-0.408532,-0.408239,...,1.123114,-0.886265,1.105498,1.007528,-0.995596,1.006938,-0.995868,-0.998166,0.996724,-1.003757


In [132]:
y_train_sme_standard_be = data_sme_standard_be['level']

In [133]:
DecisionTree(x_train_sme_standard_be, y_train_sme_standard_be, x_test_standard_be, y_test_standard_be)

/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.8/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best parameters:  OrderedDict([('criterion', 'entropy'), ('max_depth', 97), ('min_samples_leaf', 1), ('min_samples_split', 2)])

Evaluation data training: 
Macro-Averaged Recall:  1.0
Macro-Averaged Precision:  1.0
Macro-Averaged Precision:  1.0
Accuracy:  1.0

Evaluation data training: 
Macro-Averaged F1 Score:  0.5950400208800093
Macro-Averaged Recall:  0.6225908432702831
Macro-Averaged Precision:  0.5773924909050315
Accuracy:  0.7527685366631728


In [134]:
model_sme_standard = joblib.load('model_dt1.joblib')

In [136]:
joblib.dump(model_sme_standard, 'model_sme_standard.joblib')

['model_sme_standard.joblib']